# 1. 인공지능학부 교수 소개 

In [4]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# 크롤링 대상 URL (인공지능학부 교수 소개)
url = "https://aisw.jnu.ac.kr/aisw/510/subview.do"

# 웹페이지 요청 및 HTML 파싱
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# 교수 정보가 들어있는 요소 선택
items = soup.select(".artclInfo")

# 구조화된 데이터를 저장할 리스트
structured_data = []

# 각 교수별 정보 파싱
for item in items:
    # 교수 이름 추출 (div.artclTitle strong)
    name_element = item.select_one(".artclTitle strong")
    professor_name = name_element.get_text(strip=True).replace("\n", "").replace("\t", "") if name_element else "정보 없음"

    # 필드명과 값을 추출
    dt_elements = item.select("dt")  # 필드명
    dd_elements = item.select("dd")  # 값

    # 필드명과 값을 매칭하여 딕셔너리 생성
    professor_info = {"이름": professor_name}  # 교수 이름 추가
    for dt, dd in zip(dt_elements, dd_elements):
        key = dt.get_text(strip=True)  # 필드명
        value = dd.get_text(strip=True)  # 필드값

        if len(value) <= 0:
            value = "정보 없음"

        # 홈페이지의 경우 <a> 태그의 href 값 추출
        if "홈페이지" in key:
            homepage_link = dd.select_one("a")
            value = homepage_link["href"] if homepage_link else "정보 없음"

        professor_info[key] = value

    structured_data.append(professor_info)

# CSV 파일 저장
df = pd.DataFrame(structured_data)
df.to_csv("인공지능학부 교수 소개.csv", encoding="utf-8-sig", index=False)

In [5]:
df.head()

,이름,직위(직급),연구실,전공 및 연구,연락처,이메일,홈페이지
0,"임형석(Lim, Hyeong-Seok)",교수,계산이론연구실,알고리즘,062-530-3426,hslim@jnu.ac.kr,http://jnualgo.wixsite.com/algo
1,"최덕재(Choi, Deok-Jai)",명예교수,차세대네트워크연구실,컴퓨터네트워크,정보 없음,dchoi@jnu.ac.kr,정보 없음
2,"김수형(Kim, Soo-Hyung)",교수,패턴인식연구실,인공지능,062-530-3430,shkim@jnu.ac.kr,http://pr.jnu.ac.kr/shkim/
3,"박혁로(Park, Hyuk-Ro)",교수,정보검색연구실,정보검색,062-530-3431,hyukro@jnu.ac.kr,정보 없음
4,"양형정(Yang, Hyung-Jeong)",교수,스마트컴퓨팅연구실,응용소프트웨어,062-530-3436,hjyang@jnu.ac.kr,http://sclab.jnu.ac.kr/


# 2. 인공지능학부 소개

In [7]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# 크롤링 대상 URL 
url = "https://aisw.jnu.ac.kr/aisw/509/subview.do"

# 웹페이지 요청 및 HTML 파싱
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
# 내용이 있는 inner div 찾기
content_div = soup.select_one(".inner .text")

# 데이터 저장용 딕셔너리
structured_data = {
    "학부 소개": "",
    "교육 목표 및 비전": [],
    "졸업 후 진로": [],
    "참여 사업": []
}

# 학부 소개 텍스트 추출
intro_title = content_div.select_one("p.title")
intro_content = content_div.select("p.cont")

if intro_title:
    structured_data["학부 소개"] = intro_title.get_text(strip=True)
if intro_content:
    structured_data["학부 소개"] += " " + " ".join([p.get_text(strip=True) for p in intro_content])

# 교육 목표 및 비전 리스트 추출
edu_vision_list = content_div.select("p.title.mt50 + ul")[0].select("li")
structured_data["교육 목표 및 비전"] = [li.get_text(strip=True) for li in edu_vision_list]

# 졸업 후 진로 리스트 추출
career_list = content_div.select("p.title.mt50 + ul")[1].select("li")
structured_data["졸업 후 진로"] = [li.get_text(strip=True) for li in career_list]

# 참여 사업 리스트 및 링크 추출
business_list = content_div.select("p.title.mt50 + ul")[2].select("li")
structured_data["참여 사업"] = []
for li in business_list:
    link = li.select_one("a")
    if link:
        structured_data["참여 사업"].append(f"{li.get_text(strip=True)} ({link['href']})")
    else:
        structured_data["참여 사업"].append(li.get_text(strip=True))

# CSV 변환
df = pd.DataFrame([structured_data])
df.to_csv("인공지능학부 학부 소개.csv", encoding = "utf-8-sig", index = False)

In [82]:
df

,학부 소개,교육 목표 및 비전,졸업 후 진로,참여 사업
0,인공지능학부는 소프트웨어를 기반으로 다양한 지능형 응용 프로그램 개발에 대해 교육하...,"[탄탄한 이론 및 실무 지식을 갖춘 인재 양성, 전문가로서의 책임감과 윤리성을 갖춘...","[IT :삼성, LG, NAVER, DAUM 등, 금융 :은행, 증권사, 신용보증기...","[소프트웨어중심대학사업홈페이지 (https://www.sojoong.kr/), 인공..."


# 3. 인공지능학부 교통편 안내

In [83]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# 크롤링 대상 URL 
url = "https://aisw.jnu.ac.kr/aisw/511/subview.do"

# 웹페이지 요청 및 HTML 파싱
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [98]:
soup.select("tr")

[<tr>
 <th scope="row">고속버스 시외버스편<br/>(광천동 종합터미널) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 15분 소요, 요금은 교통 체증여부에 따라 다르나 5,000원 정도임</li>
 <li>시내버스 : 광천동 종합버스터미널에서 목적지 정류소에 맞는 버스에 탑승 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 풍암26, 첨단30, 상무64, 518</li>
 <li>전남대후문 정류소 : 일곡38</li>
 </ul> </li>
 </ul> </td>
 </tr>,
 <tr>
 <th scope="row">철도편(광주역) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 5분 거리에 있으며, 요금은 3,500원 정도임</li>
 <li>시내버스 : 광주역에서 목적지 정류소에 맞는 버스에 탑승하여 오면 됨 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 첨단30, 금남57, 두암81(상행), 일곡 180, 518</li>
 <li>전남대후문 정류소 : 진월 07, 용봉83, 일곡180</li>
 </ul> </li>
 </ul> </td>
 </tr>,
 <tr>
 <th scope="row">철도편(송정역) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 30분 정도 소요, 요금은 교통 체증여부에 따라 다르나 15,000원 정도임</li>
 <li>시내버스 : 송정리역 정류장에서 목적지 정류소에 맞는 버스에 탑승, 1시간 10분이상 소요 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 좌석02, 송정19, 일곡 38</li>
 <li>

In [108]:
soup.select("tr")

[<tr>
 <th scope="row">고속버스 시외버스편<br/>(광천동 종합터미널) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 15분 소요, 요금은 교통 체증여부에 따라 다르나 5,000원 정도임</li>
 <li>시내버스 : 광천동 종합버스터미널에서 목적지 정류소에 맞는 버스에 탑승 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 풍암26, 첨단30, 상무64, 518</li>
 <li>전남대후문 정류소 : 일곡38</li>
 </ul> </li>
 </ul> </td>
 </tr>,
 <tr>
 <th scope="row">철도편(광주역) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 5분 거리에 있으며, 요금은 3,500원 정도임</li>
 <li>시내버스 : 광주역에서 목적지 정류소에 맞는 버스에 탑승하여 오면 됨 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 첨단30, 금남57, 두암81(상행), 일곡 180, 518</li>
 <li>전남대후문 정류소 : 진월 07, 용봉83, 일곡180</li>
 </ul> </li>
 </ul> </td>
 </tr>,
 <tr>
 <th scope="row">철도편(송정역) 이용시</th>
 <td class="align-l">
 <ul class="con-list">
 <li>택시 : 용봉동캠퍼스까지 30분 정도 소요, 요금은 교통 체증여부에 따라 다르나 15,000원 정도임</li>
 <li>시내버스 : 송정리역 정류장에서 목적지 정류소에 맞는 버스에 탑승, 1시간 10분이상 소요 
         <ul class="con-list2 mt5">
 <li>전남대 정류소(정문) : 좌석02, 송정19, 일곡 38</li>
 <li>

In [115]:
# soup.select("tr") 결과에서 유의미한 정보 추출
structured_data = []

for row in soup.select("tr"):
    departure = row.select_one("th").get_text(strip=True) if row.select_one("th") else "정보 없음"

    # 택시 및 시내버스 정보 추출
    transport_modes = []
    duration = "정보 없음"
    fare = "정보 없음"
    bus_numbers = []

    for li in row.select("td .con-list > li"):
        text = li.get_text(strip=True)

        # 택시 정보 추출
        if "택시" in text:
            transport_modes.append("택시")
            if "분" in text:
                duration = text.split("소요")[0].split("택시 : ")[-1].strip()
            if "원" in text:
                fare = text.split("요금은")[-1].split("정도")[0].strip()

        # 시내버스 정보 추출
        elif "시내버스" in text:
            transport_modes.append("시내버스")

        # 버스 번호 추출
        bus_list = li.select("ul.con-list2 li")
        for bus in bus_list:
            bus_numbers.append(bus.get_text(strip=True))

    structured_data.append({
        "출발지": departure,
        "이동 수단": ", ".join(transport_modes),
        "소요 시간": duration,
        "요금": fare,
        "버스 번호": ", ".join(bus_numbers) if bus_numbers else "정보 없음"
    })

# CSV 파일 저장
df = pd.DataFrame(structured_data)
df


,출발지,이동 수단,소요 시간,요금,버스 번호
0,고속버스 시외버스편(광천동 종합터미널) 이용시,"택시, 시내버스",용봉동캠퍼스까지 15분,"교통 체증여부에 따라 다르나 5,000원","전남대 정류소(정문) : 풍암26, 첨단30, 상무64, 518, 전남대후문 정류소..."
1,철도편(광주역) 이용시,"택시, 시내버스","용봉동캠퍼스까지 5분 거리에 있으며, 요금은 3,500원 정도임","3,500원","전남대 정류소(정문) : 첨단30, 금남57, 두암81(상행), 일곡 180, 51..."
2,철도편(송정역) 이용시,"택시, 시내버스",용봉동캠퍼스까지 30분 정도,"교통 체증여부에 따라 다르나 15,000원","전남대 정류소(정문) : 좌석02, 송정19, 일곡 38, 전남대후문 정류소 : 1..."
3,항공편(광주공항) 이용시,"택시, 시내버스",용봉동캠퍼스까지 택시로 약 30~40분,"교통체증 여부에 따라 다르나 약 15,000원",공항버스를 타고 광주-송정간 도로에서 내려 일곡 38번 버스를 타면 전남대후문 정류...
4,자동차편 이용시,,정보 없음,정보 없음,정보 없음
5,시내버스편 이용시,,정보 없음,정보 없음,정보 없음


In [116]:
df.insert(5, '기타', ['정보 없음', '정보 없음', '정보 없음', '정보 없음음', 
                    "고속도로를 이용하여 용봉동캠퍼스에 올 경우 서광주IC와 동광주IC 사이에 있는 용봉IC로 빠져 나온 뒤 찾아오시는 길 안내의 약도에 따라 운행하면 10분 정도 소요\
용봉IC는 출구만 있고, 입구는 없기 때문에 고속도로를 타기 위해서는 전남대 정문에서 신안3거리를 지나 운암고가 밑에서 우회전 하여 광주문화예술회관으로 나간 뒤 서광주IC를 통해 고속도로로 나감",
"전남대 정류소(정문)는 풍암26, 첨단30, 금남 57, 상무64, 두암81(상행), 일곡180, 518 버스가 정차함\
전남대후문 정류소는 진월07, 문흥18, 일곡 180, 용전 184, 송정19, 일곡28, 일곡38, 문흥80, 용봉83, 충효187, 419 버스가 정차함"])

In [117]:
df

,출발지,이동 수단,소요 시간,요금,버스 번호,기타
0,고속버스 시외버스편(광천동 종합터미널) 이용시,"택시, 시내버스",용봉동캠퍼스까지 15분,"교통 체증여부에 따라 다르나 5,000원","전남대 정류소(정문) : 풍암26, 첨단30, 상무64, 518, 전남대후문 정류소...",정보 없음
1,철도편(광주역) 이용시,"택시, 시내버스","용봉동캠퍼스까지 5분 거리에 있으며, 요금은 3,500원 정도임","3,500원","전남대 정류소(정문) : 첨단30, 금남57, 두암81(상행), 일곡 180, 51...",정보 없음
2,철도편(송정역) 이용시,"택시, 시내버스",용봉동캠퍼스까지 30분 정도,"교통 체증여부에 따라 다르나 15,000원","전남대 정류소(정문) : 좌석02, 송정19, 일곡 38, 전남대후문 정류소 : 1...",정보 없음
3,항공편(광주공항) 이용시,"택시, 시내버스",용봉동캠퍼스까지 택시로 약 30~40분,"교통체증 여부에 따라 다르나 약 15,000원",공항버스를 타고 광주-송정간 도로에서 내려 일곡 38번 버스를 타면 전남대후문 정류...,정보 없음음
4,자동차편 이용시,,정보 없음,정보 없음,정보 없음,고속도로를 이용하여 용봉동캠퍼스에 올 경우 서광주IC와 동광주IC 사이에 있는 용봉...
5,시내버스편 이용시,,정보 없음,정보 없음,정보 없음,"전남대 정류소(정문)는 풍암26, 첨단30, 금남 57, 상무64, 두암81(상행)..."


In [118]:
df.to_csv("인공지능학부 오시는길.csv", encoding ='utf-8', index = False)

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

def crawl(url, domain, visited=None):
    if visited is None:
        visited = set()

    # 이미 방문한 URL은 다시 크롤링하지 않도록 체크
    if url in visited:
        return
    visited.add(url)

    print(f"크롤링 중: {url}")

    try:
        # HTTP GET 요청 보내기 (필요시 headers 등을 추가할 수 있음)
        response = requests.get(url)
        if response.status_code != 200:
            print(f"페이지 요청 실패: {url} (상태 코드: {response.status_code})")
            return

        # HTML 소스 코드 가져오기
        html_content = response.text

        # BeautifulSoup으로 HTML 파싱하기
        soup = BeautifulSoup(html_content, 'html.parser')

        # 모든 <a> 태그의 href 속성 추출 (내부 링크 위주로 처리)
        for a_tag in soup.find_all('a'):
            href = a_tag.get('href')
            if not href:
                continue

            # 상대 경로인 경우, 절대 URL로 변환
            full_url = urljoin(url, href)
            parsed_url = urlparse(full_url) 

            # 동일 도메인 내 링크만 처리 
            if parsed_url.netloc == domain:
                visited.add(full_url)
                # 필요에 따라 재귀적으로 크롤링할 수 있음 (현재는 주석 처리)
                # crawl(full_url, visited)
        return list(visited)
    except Exception as e:
        print(f"오류 발생: {e}")

In [83]:
"""노가다"""
link = "https://mathedu.jnu.ac.kr/mathedu/15864/subview.do" 
domain = "mathedu.jnu.ac.kr"

URLS = crawl(link, domain)

for url in URLS:
    if "rss" in url:
        rss_url = url
        break 

크롤링 중: https://mathedu.jnu.ac.kr/mathedu/15864/subview.do


In [84]:
rss_url = rss_url[ :len(rss_url) - 2] 
number = "3000"
rss_url = rss_url + number

In [85]:
print(rss_url)

https://mathedu.jnu.ac.kr/bbs/mathedu/2306/rssList.do?row=3000


In [86]:
import feedparser
import pandas as pd

feed = feedparser.parse(rss_url)

"""노가다"""
department = "수학교육과"

# 데이터를 리스트로 저장
entries = []
for entry in feed.entries:
    entries.append({
        "학부": department,
        "제목": entry.title,
        "링크": entry.link,
        "내용": entry.summary,
        "작성일": entry.published
    })

# 데이터프레임 변환
df = pd.DataFrame(entries)

# CSV 파일로 저장
df.to_csv(f"./사범대학/{department} rss_feed.csv", index=False, encoding="utf-8-sig")

# JSON 파일로 저장
df.to_json(f"./사범대학/{department} rss_feed.json", orient="records", force_ascii=False)

print("CSV 및 JSON 파일 저장 완료!")
    

CSV 및 JSON 파일 저장 완료!
